## Object Detection

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

In [6]:
# Path dan kelas
image_dir = "dataset/object_detection/test/images"
label_dir = "dataset/object_detection/test/labels"
model_path = "results/object_detection/training/nano/weights/best.pt"
class_names = ['leaf_healthy', 'leaf_rust', 'leaf_scab']
iou_threshold = 0.5
max_display = 5  # batasi hanya tampilkan maksimal 5 gambar mismatch

# Load model
model = YOLO(model_path)

# Fungsi IoU
def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea != 0 else 0

# Ambil semua gambar
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))])

displayed = 0
for image_name in image_files:
    if displayed >= max_display:
        break

    # Path gambar dan label
    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + ".txt")

    # Load gambar
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    gt_image = image.copy()
    pred_image = image.copy()

    # --- Ground Truth boxes ---
    gt_boxes = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                cls, x_center, y_center, bw, bh = map(float, line.strip().split())
                x1 = int((x_center - bw / 2) * w)
                y1 = int((y_center - bh / 2) * h)
                x2 = int((x_center + bw / 2) * w)
                y2 = int((y_center + bh / 2) * h)
                gt_boxes.append((x1, y1, x2, y2, int(cls)))

    # --- Prediction boxes ---
    results = model.predict(image_path, conf=0.25, iou=0.5, verbose=False)[0]
    pred_boxes = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        pred_boxes.append((x1, y1, x2, y2, cls, conf))

    # --- Matching logic ---
    matched_gt = set()
    matched_pred = set()

    for i, gt in enumerate(gt_boxes):
        for j, pred in enumerate(pred_boxes):
            iou = compute_iou(gt[:4], pred[:4])
            if iou >= iou_threshold and gt[4] == pred[4]:
                matched_gt.add(i)
                matched_pred.add(j)
                break

    fn = [i for i in range(len(gt_boxes)) if i not in matched_gt]
    fp = [j for j in range(len(pred_boxes)) if j not in matched_pred]

    if not fn and not fp:
        continue  # skip karena semuanya match

    # --- Visualisasi jika mismatch ditemukan ---
    for i, (x1, y1, x2, y2, cls) in enumerate(gt_boxes):
        color = (255, 0, 0) if i not in fn else (0, 0, 255)  # merah = FN
        label = class_names[cls] + (" (FN)" if i in fn else "")
        cv2.rectangle(gt_image, (x1, y1), (x2, y2), color, 2)
        # Hitung posisi teks, sesuaikan agar tidak keluar dari frame
        label_y = y1 - 10 if y1 - 10 > 10 else y1 + 20
        cv2.putText(gt_image, label, (x1, label_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)



    for j, (x1, y1, x2, y2, cls, conf) in enumerate(pred_boxes):
        color = (0, 255, 0) if j not in fp else (255, 255, 0)  # kuning = FP
        label = f"{class_names[cls]} {conf:.2f}" + (" (FP)" if j in fp else "")
        cv2.rectangle(pred_image, (x1, y1), (x2, y2), color, 2)
        label_y = y1 - 10 if y1 - 10 > 10 else y1 + 20
        cv2.putText(pred_image, label, (x1, label_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)



    # Gabung dan tampilkan
    combined = np.concatenate((gt_image, pred_image), axis=1)
    plt.figure(figsize=(12, 6))
    # Konversi kembali ke BGR untuk penyimpanan
    combined_bgr = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)

    # Buat folder output
    os.makedirs("DELETE", exist_ok=True)
    cv2.imwrite(f"DELETE/{os.path.splitext(image_name)[0]}_mismatch.jpg", combined_bgr)

    displayed += 1


<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

## Mask Daun

In [10]:
# Paths
image_dir = "dataset/mask_daun/test/images"
label_dir = "dataset/mask_daun/test/labels"
model_path = "results/mask_daun/training/nano/weights/best.pt"
output_dir = "DELETE"
os.makedirs(output_dir, exist_ok=True)

class_names = ['leaf_healthy', 'leaf_rust', 'leaf_scab']
iou_threshold = 0.5
max_display = 5

model = YOLO(model_path)

def mask_iou(mask1, mask2):
    inter = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return inter / union if union else 0

image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))])
displayed = 0

for image_name in image_files:
    if displayed >= max_display:
        break

    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + ".txt")

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape

    gt_mask_bin = np.zeros((h, w), dtype=np.uint8)
    pred_mask_bin = np.zeros((h, w), dtype=np.uint8)

    gt_vis = image_rgb.copy()
    pred_vis = image_rgb.copy()

    # Ground Truth
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for i, line in enumerate(f):
                parts = line.strip().split()
                cls = int(parts[0])
                points = list(map(float, parts[1:]))
                poly = np.array(points).reshape(-1, 2)
                poly[:, 0] *= w
                poly[:, 1] *= h
                poly = poly.astype(np.int32)
                cv2.fillPoly(gt_mask_bin, [poly], 255)

                # Overlay filled polygon
                overlay = gt_vis.copy()
                cv2.fillPoly(overlay, [poly], color=(255, 0, 0))
                gt_vis = cv2.addWeighted(overlay, 0.4, gt_vis, 0.6, 0)

                # Label posisi tengah
                M = cv2.moments(poly)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    label_text = class_names[cls]
                    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    x_text = max(0, min(cx - tw // 2, w - tw))
                    y_text = max(th + 2, min(cy + th // 2, h - 2))

                    cv2.rectangle(gt_vis, (x_text - 2, y_text - th - 2), (x_text + tw + 2, y_text + 2), (255, 255, 255), -1)
                    cv2.putText(gt_vis, label_text, (x_text, y_text), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Prediction
    results = model.predict(image_path, conf=0.25, iou=0.5, verbose=False, retina_masks=True)[0]
    for i, seg in enumerate(results.masks.data):
        mask = seg.cpu().numpy().astype(np.uint8)
        pred_mask_bin[mask > 0] = 255
        cls = int(results.boxes.cls[i])

        # Contour
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        overlay = pred_vis.copy()
        cv2.drawContours(overlay, contours, -1, (0, 255, 0), -1)
        pred_vis = cv2.addWeighted(overlay, 0.4, pred_vis, 0.6, 0)

        # Label
        M = cv2.moments(mask)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            label_text = class_names[cls]
            (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            x_text = max(0, min(cx - tw // 2, w - tw))
            y_text = max(th + 2, min(cy + th // 2, h - 2))

            cv2.rectangle(pred_vis, (x_text - 2, y_text - th - 2), (x_text + tw + 2, y_text + 2), (255, 255, 255), -1)
            cv2.putText(pred_vis, label_text, (x_text, y_text), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Hitung IoU
    iou = mask_iou(gt_mask_bin > 0, pred_mask_bin > 0)
    if iou >= iou_threshold:
        continue

    # Gabung dan simpan
    gt_bgr = cv2.cvtColor(gt_vis, cv2.COLOR_RGB2BGR)
    pred_bgr = cv2.cvtColor(pred_vis, cv2.COLOR_RGB2BGR)
    combined = np.concatenate((gt_bgr, pred_bgr), axis=1)

    save_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_mismatch.jpg")
    cv2.imwrite(save_path, combined)

    # plt.figure(figsize=(12, 6))
    # plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    # plt.title(f"Mismatch (IoU={iou:.2f}): {image_name} | GT (merah) vs Pred (hijau)")
    # plt.axis("off")
    # plt.show()

    displayed += 1

## Mask Lesi

In [14]:
# Paths
image_dir = "dataset/mask_lesi/valid/images"
label_dir = "dataset/mask_lesi/valid/labels"
model_path = "results/mask_lesi/training/medium/weights/best.pt"
output_dir = "DELETE"
os.makedirs(output_dir, exist_ok=True)

class_names = ['lesion_rust', 'lesion_scab']
iou_threshold = 0.5
max_display = 5

model = YOLO(model_path)

def mask_iou(mask1, mask2):
    inter = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return inter / union if union else 0

image_files = sorted([f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png"))])
displayed = 0

for image_name in image_files:
    if displayed >= max_display:
        break

    image_path = os.path.join(image_dir, image_name)
    label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + ".txt")

    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape

    gt_mask_bin = np.zeros((h, w), dtype=np.uint8)
    pred_mask_bin = np.zeros((h, w), dtype=np.uint8)

    gt_vis = image_rgb.copy()
    pred_vis = image_rgb.copy()

    # Ground Truth
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for i, line in enumerate(f):
                parts = line.strip().split()
                cls = int(parts[0])
                points = list(map(float, parts[1:]))
                poly = np.array(points).reshape(-1, 2)
                poly[:, 0] *= w
                poly[:, 1] *= h
                poly = poly.astype(np.int32)
                cv2.fillPoly(gt_mask_bin, [poly], 255)

                # Overlay filled polygon
                overlay = gt_vis.copy()
                cv2.fillPoly(overlay, [poly], color=(255, 0, 0))
                gt_vis = cv2.addWeighted(overlay, 0.4, gt_vis, 0.6, 0)

                # Label posisi tengah
                M = cv2.moments(poly)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    label_text = class_names[cls]
                    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    x_text = max(0, min(cx - tw // 2, w - tw))
                    y_text = max(th + 2, min(cy + th // 2, h - 2))

                    cv2.rectangle(gt_vis, (x_text - 2, y_text - th - 2), (x_text + tw + 2, y_text + 2), (255, 255, 255), -1)
                    cv2.putText(gt_vis, label_text, (x_text, y_text), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Prediction
    results = model.predict(image_path, conf=0.25, iou=0.5, verbose=False, retina_masks=True)[0]
    for i, seg in enumerate(results.masks.data):
        mask = seg.cpu().numpy().astype(np.uint8)
        pred_mask_bin[mask > 0] = 255
        cls = int(results.boxes.cls[i])

        # Contour
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        overlay = pred_vis.copy()
        cv2.drawContours(overlay, contours, -1, (0, 255, 0), -1)
        pred_vis = cv2.addWeighted(overlay, 0.4, pred_vis, 0.6, 0)

        # Label
        M = cv2.moments(mask)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            label_text = class_names[cls]
            (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            x_text = max(0, min(cx - tw // 2, w - tw))
            y_text = max(th + 2, min(cy + th // 2, h - 2))

            cv2.rectangle(pred_vis, (x_text - 2, y_text - th - 2), (x_text + tw + 2, y_text + 2), (255, 255, 255), -1)
            cv2.putText(pred_vis, label_text, (x_text, y_text), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Hitung IoU
    iou = mask_iou(gt_mask_bin > 0, pred_mask_bin > 0)
    # Uncomment jika hanyak ingin missmatch
    if iou >= iou_threshold:
        continue

    # Gabung dan simpan
    gt_bgr = cv2.cvtColor(gt_vis, cv2.COLOR_RGB2BGR)
    pred_bgr = cv2.cvtColor(pred_vis, cv2.COLOR_RGB2BGR)
    combined = np.concatenate((gt_bgr, pred_bgr), axis=1)

    save_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_mismatch.jpg")
    cv2.imwrite(save_path, combined)

    # plt.figure(figsize=(12, 6))
    # plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    # plt.title(f"Mismatch (IoU={iou:.2f}): {image_name} | GT (merah) vs Pred (hijau)")
    # plt.axis("off")
    # plt.show()

    displayed += 1

## Severity Estimation

In [18]:
# Path
image_dir = "dataset/object_detection/test/images"
daun_model_path = "results/mask_daun/training/medium/weights/best.pt"
lesi_model_path = "results/mask_lesi/training/medium/weights/best.pt"

# Load models
model_daun = YOLO(daun_model_path)
model_lesi = YOLO(lesi_model_path)

# Ambil 5 gambar non-healthy
image_files = [f for f in os.listdir(image_dir) if f.endswith((".jpg", ".png")) and "healthy" not in f.lower()]
image_files = sorted(image_files)[:5]

# Loop setiap gambar
for fname in image_files:
    image_path = os.path.join(image_dir, fname)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]

    # Segmentasi daun
    result_daun = model_daun.predict(image_path, verbose=False, retina_masks=True)[0]
    if result_daun.masks is None or len(result_daun.boxes) == 0:
        print(f"[SKIP] No leaf detected in: {fname}")
        continue

    max_area = 0
    chosen_daun = None
    for i, box in enumerate(result_daun.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        area = (x2 - x1) * (y2 - y1)
        if area > max_area:
            max_area = area
            chosen_daun = {
                "box": (x1, y1, x2, y2),
                "mask": result_daun.masks.data[i].cpu().numpy().astype(np.uint8)
            }

    if chosen_daun is None:
        print(f"[SKIP] No valid bounding box in: {fname}")
        continue

    # Crop ROI
    x1, y1, x2, y2 = chosen_daun["box"]
    mask_daun_full = np.zeros((h, w), dtype=np.uint8)
    mask_daun_full[chosen_daun["mask"] > 0] = 255
    mask_daun_crop = mask_daun_full[y1:y2, x1:x2]
    crop_img = image_rgb[y1:y2, x1:x2]

    # Segmentasi lesi
    result_lesi = model_lesi.predict(image_path, verbose=False, retina_masks=True)[0]
    mask_lesi_full = np.zeros((h, w), dtype=np.uint8)
    if result_lesi.masks is not None:
        for m in result_lesi.masks.data:
            mask = m.cpu().numpy().astype(np.uint8)
            mask_lesi_full[mask > 0] = 255
    mask_lesi_crop = mask_lesi_full[y1:y2, x1:x2]

    # Hitung severity
    daun_area = np.sum(mask_daun_crop > 0)
    lesi_area = np.sum(np.logical_and(mask_daun_crop > 0, mask_lesi_crop > 0))
    severity = (lesi_area / daun_area) * 100 if daun_area > 0 else 0

    # Overlay
    crop_overlay = crop_img.copy()
    crop_overlay[mask_daun_crop > 0] = cv2.addWeighted(crop_img, 0.5, np.full_like(crop_img, (0, 0, 255)), 0.5, 0)[mask_daun_crop > 0]
    crop_overlay[mask_lesi_crop > 0] = cv2.addWeighted(crop_overlay, 0.5, np.full_like(crop_img, (0, 255, 0)), 0.5, 0)[mask_lesi_crop > 0]

    # Label severity
    label_text = f"{severity:.2f}%"
    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.8, 2)
    cv2.rectangle(crop_overlay, (5, 5), (5 + tw + 10, 5 + th + 10), (255, 255, 255), -1)
    cv2.putText(crop_overlay, label_text, (10, 10 + th), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

    # Tampilkan
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    axs[0].imshow(image_rgb)
    axs[0].set_title(f"Original Image: {fname}")
    axs[0].axis("off")

    axs[1].imshow(crop_overlay)
    axs[1].set_title("Segmented Leaf with Severity")
    axs[1].axis("off")

    # plt.tight_layout()
    # plt.show()

    output_dir = "DELETE"
    os.makedirs(output_dir, exist_ok=True)
    save_path = os.path.join(output_dir, f"{os.path.splitext(fname)[0]}_severity.jpg")
    fig.savefig(save_path, dpi=300)
    plt.close(fig)